In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [18]:
df_all = pd.read_excel('data.xlsx')
col2drop = ['NO','Name','SequenceAmount','Sequence','Sites Number','Binding Sites']
X = df_all.drop(col2drop,axis=1)
y = df_all['Binding Sites']  
# X.head()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [20]:
X_train_isbs = X_train.loc[y_train == 1]
X_train_notbs = X_train.loc[y_train == 0]
# X_train_isbs.head()

In [21]:
miu_isbs = X_train_isbs.describe().loc['mean']
miu_notbs = X_train_notbs.describe().loc['mean']
# miu_isbs

In [22]:
sum_isbs = np.zeros([16,16])
p_train_isbs = X_train_isbs.shape[0]
for i in range(16):
    for j in range(16):
        ti = X_train_isbs.iloc[:,i].to_numpy()
        miui = np.full((p_train_isbs),miu_isbs.iat[i])
        tj = X_train_isbs.iloc[:,j].to_numpy()
        miuj = np.full((p_train_isbs),miu_isbs.iat[j])
        sum_isbs[i][j] = np.mean((ti - miui)*(tj - miuj))
# sum_isbs

In [23]:
sum_notbs = np.zeros([16,16])
p_train_notbs = X_train_notbs.shape[0]
for i in range(16):
    for j in range(16):
        ti = X_train_notbs.iloc[:,i].to_numpy()
        miui = np.full((p_train_notbs),miu_notbs.iat[i])
        tj = X_train_notbs.iloc[:,j].to_numpy()
        miuj = np.full((p_train_notbs),miu_notbs.iat[j])
        sum_notbs[i][j] = np.mean((ti - miui)*(tj - miuj))
# sum_notbs

In [24]:
delta_isbs = []
p_test = X_test.shape[0]
for i in range(p_test):
    tmp = (X_test.iloc[i] - miu_isbs).to_numpy()
    tmp_t = np.transpose(tmp)
    sum_isbs_inv = np.linalg.inv(sum_isbs)
    delta_isbs.append(tmp_t.dot(sum_isbs_inv).dot(tmp))
# len(delta_isbs)



In [25]:
delta_notbs = []
for i in range(p_test):
    tmp = (X_test.iloc[i] - miu_notbs).to_numpy()
    tmp_t = np.transpose(tmp)
    sum_notbs_inv = np.linalg.inv(sum_notbs)
    delta_notbs.append(tmp_t.dot(sum_notbs_inv).dot(tmp))
# len(delta_notbs)

In [26]:
def sigmoid_pos(x):
    s = 1 / (1 + np.exp(-x)) + 1
    return s

In [27]:
lnp_train_isbs = np.full((p_test),sigmoid_pos(p_train_isbs))
lnsum_isbs_det = np.full((p_test),sigmoid_pos(np.linalg.det(sum_isbs)))
delta_isbs = np.array(delta_isbs)
eta_isbs = lnp_train_isbs - delta_isbs/2 - lnsum_isbs_det/2
# len(eta_isbs)

In [28]:
lnp_train_notbs = np.full((p_test),sigmoid_pos(p_train_notbs))
lnsum_notbs_det = np.full((p_test),sigmoid_pos(np.linalg.det(sum_notbs)))
delta_notbs = np.array(delta_notbs)
eta_notbs = lnp_train_notbs - delta_notbs/2 - lnsum_notbs_det/2
# len(eta_notbs)

In [29]:
y_pred = eta_isbs > eta_notbs
# y_pred

In [30]:
accuracy_score(y_test, y_pred)

0.5471246006389776